# Mixed Likelihood GPLVM

In [ ]:
# If project root is not in sys.path (i.e. not installed with pip), we need to add it manually
import sys
import os
gpdir = os.path.abspath("../..")
sys.path.append(gpdir)

In [ ]:
import time

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline

In [ ]:
try:
    import seaborn as sns
    sns.set()
    print("Using Seaborn plotting")
except ModuleNotFoundError:
    print("Using Matplotlib plotting")

In [ ]:
import gp
from gp.util.data import oilflow, circle_data, gaussian_data
from gp.model import MLGPLVM

## Generate data

In [ ]:
np.random.seed(1)

In [ ]:
num_data = 500
latent_dim = 2
y_obs, likelihoods, labels = oilflow(num_data)
x = gp.util.PCA_reduce(y_obs, latent_dim)
y = tf.convert_to_tensor(y_obs, dtype=tf.float32)

## Create model

In [ ]:
kernel = gp.kernel.ARDRBF(variance=0.5, gamma=0.5, xdim=2, name="kernel")
m = MLGPLVM(y, latent_dim, x=x, kernel=kernel, likelihoods=likelihoods)

## Build graph

In [ ]:
loss = m.loss()
learning_rate = 1e-4
with tf.name_scope("train"):
    trainable_vars = tf.trainable_variables()
    train_all = tf.train.RMSPropOptimizer(learning_rate).minimize(loss, var_list=trainable_vars, name="RMSProp")

with tf.name_scope("summary"):
    m.create_summaries()
    merged_summary = tf.summary.merge_all()

init = tf.global_variables_initializer()

## Run optimisation

In [ ]:
loss_list = []
f, (ax1, ax2) = plt.subplots(1, 2)
n_iter = 10000
n_print = 100
try:
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(f"../../log/{time.strftime('%Y%m%d%H%M%S')}", sess.graph)
        print("Initializing variables...")
        sess.run(init)
        print("Starting training...")
        for i in range(n_iter):
            #sess.run(train_x)
            #sess.run(train_u)
            sess.run(train_all)
            if i % n_print == 0:
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                train_loss, summary = sess.run([loss, merged_summary], options=run_options, run_metadata=run_metadata)
                summary_writer.add_run_metadata(run_metadata, f"step{i}")
                summary_writer.add_summary(summary, i)
                x_mean = sess.run(m.qx_mean)
                z = sess.run(m.z)
                loss_list.append(train_loss)
            
                # Plot
                for c in np.unique(labels):
                    ax1.scatter(*x_mean[labels == c].T)
                # ax1.scatter(*z.T, c="k", marker="x")
                ax1.set_title(f"Step {i} - Loss: {train_loss}")
                ax2.plot(loss_list)
                display.display(f)
                display.clear_output(wait=True)
                ax1.cla()
                ax2.cla()
        x_mean = sess.run(m.qx_mean)
        z = sess.run(m.z)
        for c in np.unique(labels):
            ax1.scatter(*x_mean[labels == c].T)
        ax1.set_title(f"Step {i} - Loss: {train_loss}")
        ax2.plot(loss_list)
        display.display(f)
        display.clear_output(wait=True)
except KeyboardInterrupt:
    for c in np.unique(labels):
        ax1.scatter(*x_mean[labels == c].T)
    ax1.set_title(f"Step {i} - Loss: {train_loss}")
    ax2.plot(loss_list)
    display.display(f)
    display.clear_output(wait=True)